In [1]:
from __future__ import print_function
import time
import os
import glob
import numpy
import math
import pandas as pd
from scipy.fftpack import fft
from scipy.signal import find_peaks
from scipy.stats import skew, kurtosis 
import spectrum 
import matplotlib.pyplot as plt
from scipy.signal import lfilter

In [2]:
eps = 0.00000001
n_short_blocks = 2
c = 0.9  #threshold do spectral roll off

## Funções criadas ou modificadas para os fluxos

In [3]:
def compareToMedian(frame, median):
    """Implementação do "sign" considerando a mediana e não a linha 0"""
    result = numpy.empty(shape=(len(frame),))
    for i in range(len(frame)):
        if frame[i] > median:
            result[i] = 1
        elif frame[i] < median:
            result[i] = -1
        elif frame[i] == median:
            result[i] = 0
        else:
            result[i] = math.nan
    return result

In [4]:
def median_abs(flow, median):
    """Implementação do "abs" considerando a mediana e não a linha 0"""
    result = numpy.empty(shape=(len(flow),))
    
    for i in range(len(flow)):
        result[i] = abs(flow[i] - median) 
        
    return result

## Time-domain features

In [5]:
def stMedianCR(frame, median):
    """Computes median crossing rate of frame"""
    count = len(frame)
    countM = numpy.sum(numpy.abs(numpy.diff(compareToMedian(frame, median)))) / 2
    return (numpy.float64(countM) / numpy.float64(count-1.0))

In [6]:
def stEnergy(frame):
    """Computes signal energy of frame"""
    return numpy.sum(frame ** 2) / numpy.float64(len(frame))

In [7]:
def flowStEnergyEntropy(frame, n_short_blocks=2):  
    
    """Computes entropy of energy"""
    L = len(frame)
    Eol = numpy.sum(frame ** 2)    # total frame energy
    sub_win_len = int(numpy.floor(L / n_short_blocks))  
    if L != sub_win_len * n_short_blocks:
            frame = frame[0:sub_win_len * n_short_blocks]  
    # sub_wins is of size [n_short_blocks x L]
    sub_wins = frame.reshape(sub_win_len, n_short_blocks, order='F').copy()

    # Compute normalized sub-frame energies:
    s = numpy.sum(sub_wins ** 2, axis=0) / (Eol + eps)

    # Compute entropy of the normalized sub-frame energies:
    Entropy = round(-numpy.sum(s * numpy.log2(s + eps)),10)
    return Entropy

In [8]:
def stLPMCRatio(lp_frames, lp_median, frameMCRate):
    lp_MCRate = stMedianCR(lp_frames, lp_median)
   
    return frameMCRate/(lp_MCRate + eps)

In [9]:
def amplitudedescriptors(flow):
    abs_flow = median_abs(flow, numpy.median(flow))
    threshold = numpy.mean(abs_flow) + numpy.std(abs_flow)
    
    new_LoHAS = False
    new_LoLAS = False
    counter_LAS = 0
    counter_HAS = 0
    accumulator_AHA = 0.0
    
    list_LoHAS = []
    list_AHA = []
    list_LoLAS = []
    
    for i in range(len(abs_flow)):
        if (abs_flow[i] >= threshold and new_LoHAS):
            counter_HAS = counter_HAS + 1 #continue HAS
            accumulator_AHA = accumulator_AHA + (abs_flow[i]-threshold) #increase AHA
        
        elif (abs_flow[i] >= threshold and ~new_LoHAS):
            # new HAS
            new_LoHAS = True
            counter_HAS = 1
            # end LAS
            new_LoLAS = False
            list_LoLAS.append(counter_LAS)
            # init AHA
            accumulator_AHA = abs_flow[i]-threshold
            
        elif (abs_flow[i] < threshold and new_LoLAS):
            #continue with LAS
            counter_LAS = counter_LAS + 1

        elif (abs_flow[i] < threshold and ~new_LoLAS):
            if (new_LoHAS):
                #end HAS
                list_LoHAS.append(counter_HAS)
                new_LoHAS = False
                #end AHA
                list_AHA.append(accumulator_AHA)
            
            #new LAS
            new_LoLAS = True
            counter_LAS = 1
            
    if  list_LoHAS == []:
        lohas_mean = 0.0
        lohas_std = 0.0
        lohas_median = 0.0
    else:
        lohas_mean = numpy.mean(list_LoHAS)
        lohas_std = numpy.std(list_LoHAS)
        lohas_median = numpy.median(list_LoHAS)
        
    if  list_LoLAS == []:
        lolas_mean = 0.0
        lolas_std = 0.0
        lolas_median = 0.0
    else:
        lolas_mean = numpy.mean(list_LoLAS)
        lolas_std = numpy.std(list_LoLAS)
        lolas_median = numpy.median(list_LoLAS)
    
    if  list_AHA == []:
        aha_mean = 0.0
    else:
        aha_mean = numpy.mean(list_AHA)
    
        
    return lohas_mean, lohas_std, lohas_median, lolas_mean, lolas_std, lolas_median, aha_mean



In [10]:
def mpeg7aw(frame):
    return numpy.amin(frame), numpy.amax(frame)

In [11]:
def volume(frame):
    return numpy.sqrt(numpy.mean(frame**2))

In [12]:
def logAttackSentence(flow):
    return math.log(numpy.argmax(flow) + eps)


# Frequency-domain features

In [13]:
def stSpectralEntropy(X, n_short_blocks=n_short_blocks):
    """Computes the spectral entropy"""
    L = len(X)                         # number of frame samples
    Eol = numpy.sum(X ** 2)            # total spectral energy

    sub_win_len = int(numpy.floor(L / n_short_blocks))   # length of sub-frame
    if L != sub_win_len * n_short_blocks:
        X = X[0:sub_win_len * n_short_blocks]

    sub_wins = X.reshape(sub_win_len, n_short_blocks, order='F').copy()  # define sub-frames (using matrix reshape)
    s = numpy.sum(sub_wins ** 2, axis=0) / (Eol + eps)                      # compute spectral sub-energies
    En = -numpy.sum(s*numpy.log2(s + eps))                                    # compute spectral entropy

    return En

In [14]:
def stSpectralFlux(X, X_prev):
    """
    Computes the spectral flux feature of the current frame
    ARGUMENTS:
        X:            the abs(fft) of the current frame
        X_prev:        the abs(fft) of the previous frame
    """
    # compute the spectral flux as the sum of square distances:
    sumX = numpy.sum(X + eps)
    sumPrevX = numpy.sum(X_prev + eps)
    F = numpy.sum((X / sumX - X_prev/sumPrevX) ** 2)

    return F

In [15]:
def stSpectralRollOff(X, c):#, fs):
    """Computes spectral roll-off"""
    totalEnergy = numpy.sum(X ** 2)
    fftLength = len(X)
    Thres = c*totalEnergy
    # Ffind the spectral rolloff as the frequency position
    # where the respective spectral energy is equal to c*totalEnergy
    CumSum = numpy.cumsum(X ** 2) + eps
    [a, ] = numpy.nonzero(CumSum > Thres)
    if len(a) > 0:
        mC = numpy.float64(a[0]) / (float(fftLength))
    else:
        mC = 0.0
    return (mC)

In [16]:
def stSpectralEnergyRatio(specframe, specflow):
    return stEnergy(specframe)/(stEnergy(specflow) + eps)

In [17]:
def stPitch(frame):
    autocorrFrame, lag = spectrum.xcorr(frame, norm='biased')
    indices = find_peaks(autocorrFrame)[0]
    if len(indices) == 0:
        return 0.0
    return autocorrFrame[indices[0]]

In [18]:
def stJitter(pitches):
    diff = []
    for i in range(1,len(pitches)-1):
        diff.append(pitches[i]-pitches[i-1])
    return (abs(numpy.mean(diff)))

In [19]:
def stSpectralFlatness(frame):
    frame_power = frame ** 2
    gmean = numpy.exp(numpy.mean(numpy.log(frame_power + eps), axis=0, keepdims=True))
    amean = numpy.mean(frame_power, axis=0, keepdims=True)
    return gmean / (amean + eps)

In [20]:
def stSpectralCrestFactor(frame):
    frame_power = frame ** 2
    max = numpy.amax(frame_power)
    amean = numpy.mean(frame_power, axis=0, keepdims=True)
    return max / (amean + eps)

In [21]:
def stSpectralSkew(frame):
    return skew(frame, bias=False)

In [22]:
def stSpectralKurtosis(frame):
    return kurtosis(frame, bias=False)

## Last Frame Sentence Padding

In [24]:
def insertLastValueInFrames(flow, numFrames, k):

    numFrames = int(numFrames)

    # Signal normalization
    flow = numpy.double(flow)

    framesArray = numpy.array_split(flow, numFrames)
    
    min_num_sent_per_frame = k*2 # K*2 minimum number of sentences in a subframe
    
    padded_flow = []
    
    for i in range(len(framesArray)):
        actual_frame_array = framesArray[i]
        n = len(actual_frame_array)
        
        padded_frame = actual_frame_array
        
        if ( n < min_num_sent_per_frame): 
            if (n > 0):
                last_value = actual_frame_array[n-1]
            
            for j in range(min_num_sent_per_frame - n):
                padded_frame = numpy.append(padded_frame, last_value)
                
        padded_flow = numpy.append(padded_flow, padded_frame)#, axis=None)
        
    return padded_flow

## Flow short-term Feature Extractiton

In [25]:
def flowStFeatureExtraction(flow, flowtype, numFrames = 3):
    """
    This function implements the shor-term windowing process. For each short-term window a set of features is extracted.
    This results to a sequence of feature vectors, stored in a numpy matrix.

    ARGUMENTS
        flow:         the input flow
        flowtype:     the flow type (arg, sen, val, mod or pre)
        numFrames:    the number of frames the flow will be divided into
    RETURNS
        st_features:   a numpy array (n_feats x numOfShortTermWindows)
        feature_names: an array containing the names os the features
    """

    numFrames = int(numFrames)

    # Signal normalization
    flow = numpy.double(flow)
    
    # Median calculation - used in median crossing rate
    median = numpy.median(flow)
    N = len(flow)                                # total number of samples
    
    print(N)
    
    #preparing to the LP-MCR - linear prediction median-crossing ratio
    lp_flow = numpy.insert(flow, 0, 1.0)
    lp_flow, lp_error, lp_k= spectrum.LEVINSON(lp_flow, (len(lp_flow)-1),allow_singularity=True)
    lp_median = numpy.median(lp_flow)
    
        
    framesArray = numpy.array_split(flow, numFrames)
    lp_framesArray = numpy.array_split(lp_flow, numFrames)
    
    amplitude_descriptors = amplitudedescriptors(flow)
    
    logAttack = logAttackSentence(flow)
    
    nFFT = int(len(flow)/numFrames/2) #nftt pa usar nos frames
    
    #FFT do sinal inteiro para cálculo da spec energy ratio
    Y = abs(fft(flow))                     # get fft magnitude
    Y = Y[0:int(len(flow)/2)]                          # normalize fft 
    Y = Y/len(Y)
    
    count_fr = 0
    
    n_time_feats = 8   #median crossing rate + energy + energy_entropy    
    n_freq_feats = 9   #spectral entropy + spectral flux + spectral roll off retirei
    n_total_feats = n_time_feats + n_freq_feats                             

    feature_names = []

    st_features = []
    
    pitches = []
        
    for i in range(len(framesArray)):
        count_fr += 1
        
        #print(len(framesArray[i]))
        
        feature_names.append("frame_"+str(i)+"_"+flowtype+"_mcr")
        feature_names.append("frame_"+str(i)+"_"+flowtype+"_eng")
        feature_names.append("frame_"+str(i)+"_"+flowtype+"_eng_ent")
        feature_names.append("frame_"+str(i)+"_"+flowtype+"_spec_ent")
        feature_names.append("frame_"+str(i)+"_"+flowtype+"_spec_flux")
        feature_names.append("frame_"+str(i)+"_"+flowtype+"_spec_rolloff")
        
        feature_names.append("frame_"+str(i)+"_"+flowtype+"_lp_mcr")
        
        feature_names.append("frame_"+str(i)+"_"+flowtype+"_aw_min")
        feature_names.append("frame_"+str(i)+"_"+flowtype+"_aw_max")
        feature_names.append("frame_"+str(i)+"_"+flowtype+"_aw_diff")
        feature_names.append("frame_"+str(i)+"_"+flowtype+"_vol")
        
        
        feature_names.append("frame_"+str(i)+"_"+flowtype+"_spec_eng_ratio")
        feature_names.append("frame_"+str(i)+"_"+flowtype+"_pitch")
        feature_names.append("frame_"+str(i)+"_"+flowtype+"_spec_flatness")
        feature_names.append("frame_"+str(i)+"_"+flowtype+"_spec_crest_factor")
        feature_names.append("frame_"+str(i)+"_"+flowtype+"_spec_skew")
        feature_names.append("frame_"+str(i)+"_"+flowtype+"_spec_kurtosis")
        
        curFV = numpy.zeros((n_total_feats, 1))
        
        if len(framesArray[i]) < 1:
                     
            curFV[0] = stMedianCR(framesArray[i], median)                 # zero crossing rate
            curFV[1] = stEnergy(framesArray[i])                           # short-term energy
            curFV[2] = flowStEnergyEntropy(framesArray[i])                # short-term entropy of energy        
            curFV[3] = math.nan # stSpectralEntropy(X)
            curFV[4] = math.nan # stSpectralFlux(X, X_prev)
            curFV[5] = math.nan # stSpectralRollOff(X, c)
            
            curFV[6] = stLPMCRatio(lp_framesArray[i], lp_median, curFV[0])
            
            curFV[7], curFV[8] = mpeg7aw(framesArray[i])
            curFV[9] = curFV[8] - curFV[7] #diferença entre os valores máximo e mínimo do frame - aprox de shimmer
            
            curFV[10] = volume(framesArray[i])
            
            curFV[11] = math.nan
            curFV[12] = math.nan
            curFv[13] = math.nan
            curFv[14] = math.nan
            curFv[15] = math.nan
            curFv[16] = math.nan
            
            
        else: 
           
            X = abs(fft(framesArray[i]))                     # get fft magnitude
            X = X[0:nFFT]                                    # normalize fft #testar sem normalizar, com o X inteiro
            X = X/len(X)
            
            if count_fr == 1:
                X_prev = X.copy()                             # keep previous fft mag (used in spectral flux)
            
            curFV[0] = stMedianCR(framesArray[i], median)                 # zero crossing rate
            curFV[1] = stEnergy(framesArray[i])                           # short-term energy
            curFV[2] = flowStEnergyEntropy(framesArray[i])                # short-term entropy of energy        
            curFV[3] = stSpectralEntropy(X)
            curFV[4] = stSpectralFlux(X, X_prev)
            curFV[5] = stSpectralRollOff(X, c)
            
            curFV[6] = stLPMCRatio(lp_framesArray[i], lp_median, curFV[0])
            curFV[7], curFV[8] = mpeg7aw(framesArray[i])
            curFV[9] = curFV[8] - curFV[7] #diferença entre os valores máximo e mínimo do frame - aprox de shimmer
            curFV[10] = volume(framesArray[i])
            curFV[11] = stSpectralEnergyRatio(X, Y)
            curFV[12] = stPitch(framesArray[i])
            pitches.append(curFV[12])
            curFV[13] = stSpectralFlatness(X)
            curFV[14] = stSpectralCrestFactor(X)
            curFV[15] = stSpectralSkew(X)
            curFV[16] = stSpectralKurtosis(X)
            
            

        st_features.append(curFV)        
        X_prev = X.copy()
    
    
    feature_names.append(flowtype+"_ad_has_mean")
    feature_names.append(flowtype+"_ad_has_std")
    feature_names.append(flowtype+"_ad_has_median")
    feature_names.append(flowtype+"_ad_las_mean")
    feature_names.append(flowtype+"_ad_las_std")
    feature_names.append(flowtype+"_ad_las_median")
    feature_names.append(flowtype+"_ad_aha_mean")
    feature_names.append(flowtype+"_log_attack")
    feature_names.append(flowtype+"_jitter")
    
    jitter = stJitter(pitches)
    st_features = numpy.concatenate(st_features, axis=None)
    st_features = numpy.concatenate([st_features,amplitude_descriptors], axis=None)
    st_features = numpy.append(st_features, [logAttack,jitter])
     
    return st_features, feature_names

### Example to extract AudioLikeFeatures using a Lexicon with two dimensions: pos and neg

In [28]:
def allFlowStFeatureExtraction(pos,neg, numFrames, k):

    """Receives all flows from a text and returns their features e features names"""

    feature_names = []
    st_features = []

    cur_feature_names = []
    cur_st_features = []

    #Pos
    cur_st_features, cur_feature_names = flowStFeatureExtraction(insertLastValueInFrames(pos, numFrames, k), 'pos', numFrames)
    st_features.append(cur_st_features)
    feature_names += cur_feature_names

    #Neg
    cur_st_features, cur_feature_names = flowStFeatureExtraction(insertLastValueInFrames(neg, numFrames, k), 'neg', numFrames)
    st_features.append(cur_st_features)
    feature_names += cur_feature_names

    st_features = numpy.concatenate(st_features, axis=None)
    return st_features, feature_names


In [25]:
def readAllTextsAndGenerateFeaturesToCsv(input, output, numFrames, k):

    """Read the csv containing texts and flows, call allFlowStFeatureExtraction for each text and generate the resulting csv"""

    data = pd.read_csv(input)
    num_texts = data.shape[0]

    pos = data['positive_flow']
    pos = pos.apply(lambda s: [float(x.strip(' []')) for x in s.split(',')])
    neg = data['negative_flow']
    neg = neg.apply(lambda s: [float(x.strip(' []')) for x in s.split(',')])
    
    feature_names = []
    st_features = []

    cur_feature_names = []
    cur_st_features = []

    for i in range(num_texts):
        cur_st_features, cur_feature_names = allFlowStFeatureExtraction(pos.iat[i], neg.iat[i], numFrames, k)
        st_features.append(cur_st_features)

    feature_names += cur_feature_names

    texts = data['Comment']
    labels = data['label']
    folds = data['fold']
    

    df = pd.DataFrame(st_features, columns=feature_names)
    df.insert(loc=0, column='label', value=labels)
    df.insert(loc=1, column='fold', value=folds)
    df.insert(loc=2, column='Comment', value=texts)

   
    df.to_csv(output, index=False)
